<a href="https://colab.research.google.com/github/DAEJJY/chatgpt/blob/main/clothes_train_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#말벌에 데이터셋을 가져오기 위한 코드(roboflow에서 dataset을 제공하고 있음)
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="fNmB1Zp1Cdu4W0EN0rmQ")
project = rf.workspace("tf2bee").project("bee_train")
dataset = project.version(1).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.2 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=b191bd5463b6e0acff3fb2c7109e94c59393892e4a13b96230b91094f63f6390
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
    

loading Roboflow workspace...
loading Roboflow project...

Extracting Dataset Version Zip to Bee_train-1 in yolov5pytorch:: 100%|██████████| 2302/2302 [00:00<00:00, 2946.55it/s]


In [ ]:
#욜로 클론을 설치하는 코드 디렉토리가 생성됨
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

#중요!!! dataset 폴더를 만들기!!! 가져온 데이터셋에서 모든 자료를 새로만든 dataset 폴더로 옮겨넣어주기!!! 중요!!!

In [ ]:
#욜로5를 위한 패키지 설치
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#data.yaml 파일 확인
%cat /content/dataset/data.yaml

In [ ]:
#이미지 저장한 리스트 생성 및 이미지개수확인
%cd /
from glob import glob
img_list = glob('/content/dataset/train/images/*.jpg')

print(len(img_list))

In [ ]:
#트레이닝셋과 벨리데이션 나누어 주기
#train_test_split() 트레이닝 셋과 테스트 셋을 나누는 함수
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size = 0.2, random_state=2000)
#트레이닝 셋은 80퍼센트, 벨리데이션셋은 20퍼센트)

print(len(train_img_list),len(val_img_list))

In [ ]:
#txt로 데이터셋 저장
with open('/content/dataset/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')

with open('/content/dataset/val.txt', 'w') as f:
    f.write('\n'.join(val_img_list) + '\n')

In [ ]:
#yaml의 위치에 트레이닝셋과 밸리데이션 셋을 넣어주기
import yaml

with open('/content/dataset/data.yaml', 'r') as f:
    data = yaml.safe_load(f)

print(data)

data['train'] = '/content/dataset/train.txt'
data['val'] = '/content/dataset/val.txt'

with open('/content/dataset/data.yaml', 'w') as f:
    yaml.dump(data, f)
print(data)

In [ ]:
#트레이닝 시키기
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 50 --data /content/dataset/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name bee_yolov5s_results

In [ ]:
#텐서보드로 확인하기

%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs/ --port 8000

In [ ]:
#데이터 넣고 추론해보기
from IPython.display import Image
import os

val_img_path = val_img_list[20]
!python detect.py --weights /content/yolov5/runs/train/bee_yolov5s_results/weights/best.pt --img 416 --conf 0.5 --source "{val_img_path}"
#!python detect.py --weights /content/yolov5/runs/train/bee_yolov5s_results/weights/best.pt --img 416 --conf 0.5 --source "{val_img_path}" --view-img
#!python detect.py --weights /content/yolov5/runs/train/bee_yolov5s_results/weights/best.pt --img 416 --conf 0.5 --source "{val_img_path}" --nosave --save-txt



In [ ]:
#내가원하는 이미지 구글드라이브에서 가져와서 추론시키기
from google.colab import drive
drive.mount('/content/drive')

# 이미지 복사
img_path = '/content/drive/MyDrive/말벌.png'  # 이미지 경로를 적절하게 수정하세요
local_img_path = '/content/image.jpg'  # 복사된 이미지를 저장할 로컬 경로

!cp "{img_path}" "{local_img_path}"

# 추론 실행
!python detect.py --weights /content/yolov5/runs/train/bee_yolov5s_results/weights/best.pt --img 416 --conf 0.5 --source "{local_img_path}"

Mounted at /content/drive
detect: weights=['/content/yolov5/runs/train/bee_yolov5s_results/weights/best.pt'], source=/content/image.jpg, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-185-g2334aa7 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/image.jpg: 224x416 1 hornet, 38.7ms
Speed: 0.3ms pre-process, 38.7ms inference, 1.4ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp8


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 비디오 복사
video_path = '/content/drive/MyDrive/말벌2.mp4'  # 비디오 경로를 적절하게 수정하세요
local_video_path = '/content/말벌2.mp4'  # 복사된 비디오를 저장할 로컬 경로

!cp "{video_path}" "{local_video_path}"

# 추론 실행
!python detect.py --weights /content/yolov5/runs/train/bee_yolov5s_results/weights/best.pt --img 416 --conf 0.5 --source "{local_video_path}" --vid-stride 1  # --vid-stride는 1로 설정하여 모든 프레임을 추론하도록 합니다


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
detect: weights=['/content/yolov5/runs/train/bee_yolov5s_results/weights/best.pt'], source=/content/말벌2.mp4, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-185-g2334aa7 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/548) /content/말벌2.mp4: 256x416 1 hornet, 40.7ms
video 1/1 (2/548) /content/말벌2.mp4: 256x416 1 hornet, 5.4ms
video 1/1 (3/548) /content/말벌2.mp4: 256x416 1 hornet, 5.5ms
video 

In [ ]:
#다운로드가 잘안될때 아래는 파일을 다른 경로로 복사하고 다운로드하는 코드 예시입니다.

# 파일 복사
# 파일 복사
!cp /content/drive/MyDrive/yolov5/runs/detect/exp10/.mp4 /content/디텍팅완료.mp4

# 파일 다운로드
from google.colab import files
files.download('/content/디텍팅완료.mp4')

cp: cannot stat '/content/drive/MyDrive/yolov5/runs/detect/exp10/디텍팅완료.mp4': No such file or directory


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#학습한 데이터를 Google Drive에 복사합니다. 예를 들어, 학습한 가중치 파일인 best.pt를 복사합니다:
!cp /content/yolov5/runs/train/bee_yolov5s_results/weights/best.pt /content/drive/MyDrive/best.pt